In [1]:
import pandas as pd
from numpy import log as ln
from numpy import nan
from statsmodels.formula.api import ols
import os
import glob
from numpy import sqrt
from msilib.schema import Error
pd.options.mode.chained_assignment = None
source = os.path.abspath(os.getcwd())

In [2]:
class oes():

    def import_crsp(path,low_memory_=True):
        """Input path for a CRSP or Ivolatility file. This function will read the csv and drop 'Unnamed: 0' column. Rename 'date' to 'DataDate' and convert the data to datetime format."""
        df = pd.read_csv(path,low_memory=low_memory_)
        
        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns= 'Unnamed: 0')
        
        if 'date' in df.columns:
            df = df.rename(columns= {"date":"DataDate"})

        df.DataDate = pd.to_datetime(df.DataDate)

        return df

    def import_ivol(ticker,folder):
        old = pd.DataFrame()
        paths = glob.glob(f"{folder}{ticker}_*.csv")
        if len(paths) != 0:
            for path in paths:
                df = oes.import_crsp(path)
                old = pd.concat([old,df])
            old.sort_values("DataDate")[['UnderlyingSymbol', 'UnderlyingPrice', 'Type', 'Expiration', 'DataDate','Strike', 'Bid', 'Ask', 'Volume', 'OpenInterest']]
            return(old)
        else:
            return(pd.DataFrame())

    def dupe_check(df_crsp,path_crsp):
        """This check looks for duplicate dates in the CRSP file. If in case the number of duplicate dates is more than 8 days in an year (ie 3%), then the check will open the corresponding csv file for CRSP and IVOL database.
        Note: Do not forget to check for different ivol files in case of multiple shareclass. """
        dupes = df_crsp[df_crsp.DataDate.duplicated(keep=False)]
        if len(dupes) == 0:
            return df_crsp
        
        # if dupes.DIVAMT.isnull().values.any():
        if len(dupes) > len(df_crsp)*0.03:
            print("dupe")
            os.startfile(path_crsp)
            input("Press enter to continue...")
            corrected_df = oes.import_crsp(path_crsp)
        
        else:
            corrected_df = df_crsp.groupby('DataDate').last()
            divamt = df_crsp.groupby('DataDate').sum().DIVAMT
            corrected_df.DIVAMT = divamt
            corrected_df = corrected_df.reset_index()
        
        return(corrected_df)

    def return_check(df,path):
        df = df.sort_values(by="DataDate")
        df = df.reset_index()
        if df['RET'].iloc[0] == "C":
            df.loc[0,'RET'] = nan
        if "C" in df.RET.tolist():
            print("Ret")
            os.startfile(path)
            input("Press enter to continue...")
            df = oes.import_crsp(path)
        return(df)

    def regression(df, dtype):
        data_type = dtype + "LogRet"                                                #Adding a suffix to signify Log Returns 
        formula = data_type + " ~  Mkt_Lnrtn"                                       #Creating the formula for Regression
        years = df.index.year.unique().tolist()                                     #Creating a list of years for yearly regression

        if 2012 in years:
            years.remove(2012)

        df_reg = pd.DataFrame()                                                     #Empty df to concat yearly data to 

        for y in years:                                                             #Running a for loop for yearly regressions
            df_year = df.loc[f"{y}"]                                                #Filtering for a specific year
            try:
                if len(df_year.dropna(subset="SynthLogRet")) < 250:                 #Removing incomplete years 
                    continue
            except:
                if len(df_year.dropna(subset="CompanyLogRet")) < 250:               #Removing incomplete years 
                    continue
            fitted = ols(formula, data = df_year).fit()                             #Running the regression on the filtered year  
            explained_rtn = fitted.predict(exog = df_year)                          #Calculating the expected returns 
            df_year.loc[:,"ExplainedReturn"] = explained_rtn                        #Assigned a column to the expected returns 
            df_year.loc[:,"ResRtn"] = df_year[data_type] - explained_rtn            #Calculating Residual Returns
            df_year.loc[:,"Tstat"] = df_year.ResRtn/sqrt(fitted.scale)              #Calculating Tstat
            df_year.loc[:,"Sig"] = abs(df_year.Tstat) > 1.96                        #Checking significance
            df_year.loc[:,f"StdErr_{dtype}"] = fitted.bse[0]
            df_reg = pd.concat([df_reg,df_year])                                    #Joining the yearly regressions  
        if len(df_reg) == 0:
            return("Lack of data")
        return(df_reg)

    def blr(blr_path):
        df = pd.read_excel(blr_path, skiprows = 5).drop(columns = ["Date","PX_LAST.1","PX_LAST","Unnamed: 2"]).rename(columns={"Date.1":"DataDate","rate":"blr"})
        return(df)

    def fred(fred_path):
        df = pd.read_excel(fred_path,skiprows=10).rename(columns={"observation_date":"DataDate","DGS1":"fred"})
        df.fred = df.fred/100
        df = df.fillna(method="ffill")
        return(df)

    def marketdata(path):
        df = oes.import_crsp(path)
        df.loc[:,"Mkt_Lnrtn"] = ln(1 + df.vwretd)
        market = df[["DataDate","Mkt_Lnrtn"]].groupby("DataDate").mean()
        market.sort_index(inplace= True)
        return(market)

    def synthetic_stock(syn_df,cfacpr,fred,blr,volume_filter,oi_filter,zero_price_filter):

        if volume_filter == True:   
            syn_df = syn_df[syn_df.Volume != 0]                                     #Filtering for volume if volume_filter is True
        
        if oi_filter == True:
            syn_df = syn_df[syn_df.OpenInterest != 0]                               #Filtering for open interest if oi_filter is True

        syn_df.DataDate = pd.to_datetime(syn_df.DataDate)                           #Converting dates from str to datetime
        syn_df.Expiration = pd.to_datetime(syn_df.Expiration)                       #Converting dates from str to datetime

        syn_df = syn_df.merge(fred, on = 'DataDate', how = "left")                  #Joining fred data with df
        syn_df = syn_df.merge(blr, on = 'DataDate', how = "left")                   #Joining blr data with df
        
        syn_df.loc[:,"Tau"] = (syn_df.Expiration - syn_df.DataDate).dt.days         #Calculating Tau
        syn_df.loc[:,"YearFrac"] =  syn_df.Tau/365                                  #Converting Tau into years

        syn_df.loc[:,"BuyDiscount"] = ((1 + syn_df.fred) ** syn_df.YearFrac)        #Calculating Buy Discount
        syn_df.loc[:,"SellDiscount"] = (1 + syn_df.blr) ** syn_df.YearFrac          #Calculating Sell Discount
        
        syn_df.loc[:,"Spread"] = abs(syn_df.Ask - syn_df.Bid)
        syn_df.loc[:,"SpreadPercentage"] = syn_df.Spread/((syn_df.Bid+syn_df.Ask)/2)
        syn_df = syn_df[syn_df.SpreadPercentage < 1]

        call = syn_df[syn_df.Type == "call"].drop(columns = "Type")                 #Splitting the df into calls and puts
        call = call.rename(columns={'Last':'CallLast', 'Bid': "CallBid",\
            'Ask':"CallAsk", 'Volume':"CallVolume", 'OpenInterest':\
                'CallOpenInterest', 'IV':"CallIV", 'Delta':'CallDelta',\
                    'Gamma':'CallGamma', 'Theta':'CallTheta', 'Vega':'CallVega'})   #Renaming the columns
        
        put = syn_df[syn_df.Type == "put"].drop(columns = "Type")                   #Splitting the df into calls and puts
        put = put.rename(columns={'Last':'PutLast', 'Bid': "PutBid", \
            'Ask':"PutAsk", 'Volume':"PutVolume", 'OpenInterest':\
                'PutOpenInterest', 'IV':"PutIV", 'Delta':'PutDelta',\
                    'Gamma':'PutGamma', 'Theta':'PutTheta', 'Vega':'PutVega'})      #Renaming the columns
        
        syn_df = call.merge(put,how = "left", on=["Expiration","DataDate",\
            "Strike", "UnderlyingSymbol","UnderlyingPrice","BuyDiscount",\
                "SellDiscount","YearFrac","Tau","blr","fred"])                      #Merging the call and put df to make them parallel

        if zero_price_filter == True:                                               #Filtering for zero price quotes if zero_price_filter is True 
            syn_df = syn_df[syn_df["CallBid"] != 0]
            syn_df = syn_df[syn_df["PutBid"] != 0]
            syn_df = syn_df[syn_df["CallAsk"] != 0]
            syn_df = syn_df[syn_df["PutAsk"] != 0]

        syn_df = syn_df[syn_df["CallBid"] != 9999]
        syn_df = syn_df[syn_df["PutBid"] != 9999]
        syn_df = syn_df[syn_df["CallAsk"] != 9999]
        syn_df = syn_df[syn_df["PutAsk"] != 9999]

        abs(syn_df.CallAsk - syn_df.PutBid)/syn_df.CallAsk 

        syn_df.loc[:,"Buy"] = syn_df.Strike/syn_df.BuyDiscount                      #Calculating Buy price for the bond
        
        syn_df.loc[:,"Sell"] = syn_df.Strike/syn_df.SellDiscount                    #Calculating Sell price for the bond
        
        syn_df.loc[:,"SynthAsk"] = syn_df.CallAsk - \
            syn_df.PutBid + syn_df.Buy                                              #Calculating Synthetic stock's ask price
        
        syn_df.loc[:,"SynthBid"] = syn_df.CallBid - \
            syn_df.PutAsk + syn_df.Sell                                             #Calculating Synthetic stock's sell price
        
        syn_df.loc[:,"SynthPrice"] = (syn_df.SynthAsk + syn_df.SynthBid)/2          #Calculating Synthetic stock's price for the specific strike price on a day

        syn_df = syn_df.groupby("DataDate").mean()                                  #Calculating Synthetic stock's price on a day
        
        syn_df = syn_df.merge(cfacpr,on='DataDate',how='left')
        
        return(syn_df)

    def summarize(mkt_crsp,mkt_ivol,tic,filt):
        global com_after_cleaning,syn_after_cleaning,com_after_regression,syn_after_regression,summary_count
        mkt_crsp.loc[:,"CompanyLogRet"] = ln(mkt_crsp.RET.astype(float) + 1)
        mkt_ivol.loc[:,"SynthLogRet"] = ln(((mkt_ivol.SynthPrice*(mkt_ivol.CFACPR.shift(1)/mkt_ivol.CFACPR))/mkt_ivol.SynthPrice.shift(1)))
        com_after_cleaning += len(mkt_crsp.dropna(subset = "CompanyLogRet"))
        syn_after_cleaning += len(mkt_ivol.dropna(subset = "SynthLogRet"))

        com_reg = oes.regression(mkt_crsp,"Company")
        ivol_reg = oes.regression(mkt_ivol,"Synth")

        if type(ivol_reg) == str or type(com_reg) == str:
            print("Reg Error")
            raise Error

        com_after_regression += len(com_reg.dropna(subset = "Tstat"))
        syn_after_regression += len(ivol_reg.dropna(subset = "Tstat"))

        syn_df_reg = ivol_reg.rename(columns={"ResRtn":"SynResRtn","Tstat":"SynTstat","Sig":"SynSig","ExplainedReturn":"SynExp"}).drop(columns= "Mkt_Lnrtn")
        com_df_reg = com_reg.rename(columns={"ResRtn":"ComResRtn","Tstat":"ComTstat","Sig":"ComSig","ExplainedReturn":"ComExp"}).drop(columns = "CFACPR")
        summary = syn_df_reg.join(com_df_reg,how="right")
        summary.loc[:,"Equal"] = summary.SynSig == summary.ComSig                   #Checking if both, syn and common, Tstats are significant
        summary.loc[:,"Direction"] = (summary.SynTstat/summary.ComTstat) >= 0       #Checking if the direction of both Tstats are similar
        for i in summary.index:                                                   
            if summary.loc[i,"Equal"] == False:                                     #Redflag = True, if significance is not equal 
                summary.loc[i,"Redflag"] = 1
            
            elif summary.loc[i,"Equal"] == True and (summary.loc[i,"SynSig"]\
                == False or summary.loc[i,"ComSig"] == False):                      #Redflag = False, if significance is equal but both are not significant
                summary.loc[i,"Redflag"] = 0
            
            elif summary.loc[i,"Equal"] == True and \
                summary.loc[i,"Direction"] == False:                                #Redflag = True, if significance is equal (both significant) but are in opposite directions
                summary.loc[i,"Redflag"] = 1
            
            else:                                                                   #Redflag = False, if significance is equal and the direction is same
                summary.loc[i,"Redflag"] = 0

        summary.replace(True,1,inplace=True)
        summary.replace(False,0,inplace=True)

        summary.loc[:,"DIVAMT"] = mkt_crsp.DIVAMT
        summary.loc[:,"PERMCO"] = mkt_crsp.PERMCO
        summary.loc[:,"TICKER"] = mkt_crsp.TICKER
        summary = summary[['TICKER','PERMCO','PRC','SynthPrice','DIVAMT','SynthLogRet','CompanyLogRet','SynExp','ComExp','SynResRtn','ComResRtn','SynTstat', 'ComTstat','SynSig','ComSig','StdErr_Synth','StdErr_Company','Equal','Direction', 'Redflag']]
        print(summary.TICKER.unique()[0],": Done",len(summary.dropna(subset= "Equal")))

        summary_count += len(summary.dropna(subset= "Equal"))

        summary_path = f'{source}\\summary_{filt}'
        if not os.path.exists(summary_path):
            os.makedirs(f"{source}\\summary_{filt}")

        sum_path = f"{source}\\summary_{filt}\\{tic}_summary_{filt}.csv"
        summary.dropna(subset = "SynResRtn").to_csv(sum_path)
        return(summary)

In [3]:
glob.glob(f"{source}\\crsp_ind\\*_crsp.csv")[0]


'c:\\OES\\CFR\\crsp_ind\\10020_crsp.csv'

In [4]:
syn_before_cleaning = 0
com_before_cleaning = 0
syn_after_cleaning = 0
com_after_cleaning = 0
syn_after_regression = 0
com_after_regression = 0
summary_count = 0
com_reg_err = 0
syn_reg_err = 0
multi_tic_err = 0 
ivol_unavail_err = 0

volume_filter = False
oi_filter = False
zero_price_filter = True
filter_9999 = True
filt = "no_filter"
fred = oes.fred(f"{source}\\DGS1 1Yr Constant Mat Treasury.xlsx")
blr = oes.blr(f"{source}\\BB BLR Index.xlsx")
marketdata = oes.marketdata(f"{source}\\crsp_ind\\7_crsp.csv")          #Taking Apple as a base for full dataset, since it is the most heavily traded stock

for path_crsp in glob.glob(f"{source}\\crsp_ind\\*_crsp.csv"):
    raw_df = oes.import_crsp(path_crsp)
    tickers = raw_df.TICKER.unique().tolist()
    permco = path_crsp.split("\\")[-1].split("_")[0]


#Comment this section out if you want to include the PERMCOs with multiple tickers
    # if len(tickers) > 1:
    #     print("multi ticker")
    #     multi_tic_err += 1
    #     continue
    # tickers = tickers[0]


    mkt_crsp = pd.DataFrame(index = marketdata.index.tolist())
    mkt_ivol = pd.DataFrame(index = marketdata.index.tolist())
    print(raw_df.PERMCO.unique()[0], tickers)

    ivol_error = False

    for tic in tickers:
        tic_ivol = oes.import_ivol(tic,f"{source}\\ivol_ind\\")
        
        if len(tic_ivol) < 1:
            print(f"{tickers} ivol file unavailable")
            ivol_error = True
            ivol_unavail_err += 1
            continue
        
        tic_com_df = oes.return_check(oes.dupe_check(raw_df[raw_df.TICKER == tic].reset_index(),path_crsp),path_crsp).set_index("DataDate")
        
        com_before_cleaning += len(tic_com_df.dropna(subset = "RET"))
        cfacpr = tic_com_df.CFACPR
        tic_ivol_df = oes.synthetic_stock(tic_ivol,cfacpr,fred,blr,volume_filter,oi_filter,zero_price_filter)
        
        mkt_crsp.loc[tic_com_df.index,"PRC"] = tic_com_df.PRC
        mkt_crsp.loc[tic_com_df.index,"TICKER"] = tic_com_df.TICKER
        mkt_crsp.loc[tic_com_df.index,"RET"] = tic_com_df.RET
        mkt_crsp.loc[tic_com_df.index,"CFACPR"] = tic_com_df.CFACPR
        mkt_crsp.loc[tic_com_df.index,"PERMCO"] = tic_com_df.PERMCO
        mkt_crsp.loc[tic_com_df.index,"DIVAMT"] = tic_com_df.DIVAMT
        mkt_crsp.loc[tic_com_df.index,"Mkt_Lnrtn"] = marketdata.Mkt_Lnrtn
        
        mkt_ivol.loc[tic_com_df.index,"Mkt_Lnrtn"] = marketdata.Mkt_Lnrtn
        mkt_ivol.loc[tic_ivol_df.index,"CFACPR"] = tic_ivol_df.CFACPR
        mkt_ivol.loc[tic_ivol_df.index,"SynthPrice"] = tic_ivol_df.SynthPrice
    
    if ivol_error == True:
        continue

    mkt_crsp.loc[:,"CompanyLogRet"] = ln(mkt_crsp.RET.astype(float) + 1)
    mkt_ivol.loc[:,"SynthLogRet"] = ln(((mkt_ivol.SynthPrice*(mkt_ivol.CFACPR.shift(1)/mkt_ivol.CFACPR))/mkt_ivol.SynthPrice.shift(1)))
    com_after_cleaning += len(mkt_crsp.dropna(subset = "CompanyLogRet"))
    syn_after_cleaning += len(mkt_ivol.dropna(subset = "SynthLogRet"))

    com_reg = oes.regression(mkt_crsp,"Company")
    if type(com_reg) == str:
        print("Reg Error - com")
        com_reg_err += 1
        continue


    ivol_reg = oes.regression(mkt_ivol,"Synth")

    if type(ivol_reg) == str:
        print("Reg Error - ivol")
        syn_reg_err +=1 
        continue

    com_after_regression += len(com_reg.dropna(subset = "Tstat"))
    syn_after_regression += len(ivol_reg.dropna(subset = "Tstat"))

    syn_df_reg = ivol_reg.rename(columns={"ResRtn":"SynResRtn","Tstat":"SynTstat","Sig":"SynSig","ExplainedReturn":"SynExp"}).drop(columns= "Mkt_Lnrtn")
    com_df_reg = com_reg.rename(columns={"ResRtn":"ComResRtn","Tstat":"ComTstat","Sig":"ComSig","ExplainedReturn":"ComExp"}).drop(columns = "CFACPR")
    summary = syn_df_reg.join(com_df_reg,how="right")
    summary.loc[:,"Equal"] = summary.SynSig == summary.ComSig                   #Checking if both, syn and common, Tstats are significant
    summary.loc[:,"Direction"] = (summary.SynTstat/summary.ComTstat) >= 0       #Checking if the direction of both Tstats are similar
    for i in summary.index:                                                   
        if summary.loc[i,"Equal"] == False:                                     #Redflag = True, if significance is not equal 
            summary.loc[i,"Redflag"] = 1
        
        elif summary.loc[i,"Equal"] == True and (summary.loc[i,"SynSig"]\
            == False or summary.loc[i,"ComSig"] == False):                      #Redflag = False, if significance is equal but both are not significant
            summary.loc[i,"Redflag"] = 0
        
        elif summary.loc[i,"Equal"] == True and \
            summary.loc[i,"Direction"] == False:                                #Redflag = True, if significance is equal (both significant) but are in opposite directions
            summary.loc[i,"Redflag"] = 1
        
        else:                                                                   #Redflag = False, if significance is equal and the direction is same
            summary.loc[i,"Redflag"] = 0

    summary.replace(True,1,inplace=True)
    summary.replace(False,0,inplace=True)

    summary.loc[:,"DIVAMT"] = mkt_crsp.DIVAMT
    summary.loc[:,"PERMCO"] = mkt_crsp.PERMCO
    summary.loc[:,"TICKER"] = mkt_crsp.TICKER
    summary = summary[['TICKER','PERMCO','PRC','SynthPrice','DIVAMT','SynthLogRet','CompanyLogRet','SynExp','ComExp','SynResRtn','ComResRtn','SynTstat', 'ComTstat','SynSig','ComSig','StdErr_Synth','StdErr_Company','Equal','Direction', 'Redflag']]
    print(summary.TICKER.unique()[0],": Done",len(summary.dropna(subset= "Equal")))

    summary_count += len(summary.dropna(subset= "Equal"))

    summary_path = f'{source}\\summary_{filt}'
    if not os.path.exists(summary_path):
        os.makedirs(f"{source}\\summary_{filt}")

    sum_path = f"{source}\\summary_{filt}\\{permco}_summary_{filt}.csv"
    summary.dropna(subset = "SynResRtn").to_csv(sum_path)

counts = pd.DataFrame(index=['Common','Synth','Total'],data={"BeforeCleaning":[com_before_cleaning,syn_before_cleaning,nan],"AfterCleaning": [com_after_cleaning,syn_after_cleaning,nan],"AfterRegression":[com_after_regression,syn_after_regression,nan],"Total":[nan,nan,summary_count]})
counts.to_csv(f"{source}\\{filt}.csv")

10020 ['CSBR']
['CSBR'] ivol file unavailable
10039 ['HIFS']
['HIFS'] ivol file unavailable
10042 ['TORM']
['TORM'] ivol file unavailable
10053 ['DRL']
['DRL'] ivol file unavailable
10059 ['LWAY']
['LWAY'] ivol file unavailable
10065 ['CEC']
Reg Error - ivol
10070 ['URRE', 'WWR']
['URRE', 'WWR'] ivol file unavailable
10104 ['BTUI']
['BTUI'] ivol file unavailable
10107 ['TECH']
TECH : Done 1259
10110 ['TRS']
Reg Error - ivol
10127 ['CLRO']
['CLRO'] ivol file unavailable
10147 ['HNR']
HNR : Done 1008
1015 ['STZ']
STZ : Done 1259
10171 ['SPLS']
SPLS : Done 1008
10181 ['ISDR']
['ISDR'] ivol file unavailable
10210 ['CRUS']
CRUS : Done 1259
10218 ['FRME']
Reg Error - ivol
10224 ['SYMC']
SYMC : Done 1259
10241 ['CRH']
CRH : Done 1259
10256 ['CGNX']
CGNX : Done 1259
10288 ['NEOG']
NEOG : Done 1259
10299 ['HALL']
Reg Error - ivol
10302 ['VTR']
VTR : Done 1259
10303 ['EA']
EA : Done 1259
10323 ['DGII']
DGII : Done 1259
1033 ['COHR']
COHR : Done 1259
10358 ['ZIXI']
Reg Error - ivol
10359 ['LSCC']

In [ ]:
print(com_reg_err)
print(syn_reg_err)
print(multi_tic_err)
print(ivol_unavail_err)

In [ ]:
final = pd.DataFrame()
for path in glob.glob("C:\\OES\\summary_no_filter\\*_summary_no_filter.csv"):
    df = pd.read_csv(path).drop(columns="Unnamed: 0")
    data = pd.DataFrame({"Ticker":str(df.TICKER.unique()).replace("[","").replace("]",""),\
        "PERMCO":df.PERMCO.unique(),\
            "Datapoints":len(df),\
                "Dividend":df.DIVAMT.sum(),\
                    "Equal%":(df.Equal.sum()/len(df)),\
                        "Direction%":(df.Direction.sum()/len(df)),\
                            "RedFlag%":(df.Redflag.sum()/len(df))})
    final = pd.concat([final,data]).reset_index(drop=True)

final.to_csv("C:\\OES\\final_no_filter.csv")

In [ ]:
pd.DataFrame({"1-Year Treasury Rate":fred.set_index("DataDate").fred,"Broker Dealer Rate": blr.set_index("DataDate").blr,"Market Log Returns":marketdata.Mkt_Lnrtn}).to_csv("Exhibit_2.csv")